In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from typing import Annotated, List
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
openai_model = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
search_tool = TavilySearchResults(max_results=2)
search_tool.invoke("LangGraph가 무엇인가요?")

In [ ]:
from langchain_core.tools import tool
from langgraph.types import interrupt

@tool
def human_assist(query):
    """Human assist tool"""
    human_response = interrupt({"query": query})
    return human_response["data"]

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
llm = ChatOpenAI(model=openai_model)
tools = [search_tool, human_assist]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def chatbot(state: State):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition
tool_node = ToolNode(tools)
workflow = StateGraph(State)
workflow.add_node("chatbot", chatbot)
workflow.add_node("tools", tool_node)
workflow.add_conditional_edges("chatbot", tools_condition)
workflow.add_edge("tools", "chatbot")
workflow.add_edge(START, "chatbot")
graph = workflow.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "user123"}}

In [ ]:
from pprint import pprint
snapshot = graph.get_state(config)
if 'messages' in snapshot.values:
    pprint(snapshot.values['messages'])
else:
    print("No messages found in the snapshot.")
print(snapshot.next)

In [ ]:
user_input1 = "AI 에이전트 개발을 위한 LangGraph의 특징에 대해 설명해주세요."
state1 = {"messages": [HumanMessage(content=user_input1)]}
response1 = graph.invoke(state1, config)
print(response1["messages"][-1].content)

In [ ]:
snapshot = graph.get_state(config)
if 'messages' in snapshot.values:
    pprint(snapshot.values['messages'])
else:
    print("No messages found in the snapshot.")
print(snapshot.next)

In [ ]:
user_input2 = "AI 에이전트 개발을 위한 기술 선택에 대한 전문가의 지원이 필요해요. 지원 요청을 해도 될까요?"
state2 = {"messages": [HumanMessage(content=user_input2)]}
response2 = graph.invoke(state2, config)
print(response2["messages"][-1].content)

In [ ]:
snapshot = graph.get_state(config)
if 'messages' in snapshot.values:
    pprint(snapshot.values['messages'])
else:
    print("No messages found in the snapshot.")
print(snapshot.next)

In [ ]:
from langgraph.types import Command

human_response = (
    "네, 물론입니다. AI 에이전트 개발을 위한 기술 선택에 대한 지원을 해드리겠습니다. "
    "우선 LangGraph를 사용하는 것에 대해 어떻게 생각하시나요? "
    "LangGraph는 AI 에이전트를 개발하는 데 매우 유용한 도구입니다. "
)
human_command = Command(resume={"data": human_response})
response = graph.invoke(human_command, config)
print(response["messages"][-1].content)

In [ ]:
snapshot = graph.get_state(config)
pprint(snapshot.values['messages'])
print(snapshot.next)

In [ ]:
user_input3 = "앞서 추천해주신 기술의 시장성은 어떤가요?"
state3 = {"messages": [HumanMessage(content=user_input3)]}
response3 = graph.invoke(state3, config)
print(response3["messages"][-1].content)

In [ ]:
snapshot = graph.get_state(config)
pprint(snapshot.values['messages'])
print(snapshot.next)

In [ ]:
user_input4 = "LangGraph의 메모리 기능 추가에 대한 전문가의 지원이 필요해요."
state4 = {"messages": [HumanMessage(content=user_input4)]}
response4 = graph.invoke(state4, config)
print(response4["messages"][-1].content)

In [ ]:
snapshot = graph.get_state(config)
if 'messages' in snapshot.values:
    pprint(snapshot.values['messages'])
else:
    print("No messages found in the snapshot.")
print(snapshot.next)

In [ ]:
human_response = (
    "MemorySaver는 메모리 기반의 체크포인터로, 각 대화의 상태를 메모리에 임시로 저장하고 관리합니다. "
    "이를 통해 챗봇은 이전 대화 내용을 기억하고 다음 번 상호작용 시에도 맥락을 유지한 상태로 대화를 진행할 수 있습니다. "
    "실제 운영 환경에서는 더 영구적인 상태 관리를 위해 데이터베이스 기반 체크포인터(예: SqliteSaver 또는 PostgresSaver)를 사용하는 것이 권장됩니다."
)
human_command = Command(resume={"data": human_response})
response = graph.invoke(human_command, config)
print(response["messages"][-1].content)

In [ ]:
snapshot = graph.get_state(config)
pprint(snapshot.values['messages'])
print(snapshot.next)